## Import modules

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import time
print("Packages are Imported !!")

Packages are Imported !!


## Loading the dataset

In [2]:
df = pd.read_csv('Twitter Sentiments.csv') 
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      31962 non-null  int64 
 1   label   31962 non-null  object
 2   tweet   31962 non-null  object
dtypes: int64(1), object(2)
memory usage: 749.2+ KB


In [3]:
df.head(5) #Show first 5 rows 

,id,label,tweet
0,1,good,@user when a father is dysfunctional and is s...
1,2,good,@user @user thanks for #lyft credit i can't us...
2,3,good,bihday your majesty
3,4,good,#model i love u take with u all the time in ...
4,5,good,factsguide: society now #motivation


## Preprocessing the dataset

In [4]:
labels=df['label'] #assign label column to labels Var
labels.value_counts() 

good    29720
bad      2242
Name: label, dtype: int64

In [32]:
# convert class labels to binary values, 0 = Bad and 1 = Good
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()#.fit(labels) #Fit label encoder.
binary_labels = encoder.fit_transform(labels)#Transform labels to normalized encoding.
df['label']=binary_labels#assign new var to columns 
processed_data=df
processed_data.head(5)

,id,label,tweet,clean_tweet
0,1,1,@user when a father is dysfunctional and is s...,when father dysfunct selfish drag kid into dys...
1,2,1,@user @user thanks for #lyft credit i can't us...,thank #lyft credit caus they offer wheelchair ...
2,3,1,bihday your majesty,bihday your majesti
3,4,1,#model i love u take with u all the time in ...,#model love take with time
4,5,1,factsguide: society now #motivation,factsguid societi #motiv


In [33]:
# convert class labels to  Bad and  Good values

def decoder(arr):
    out=list()
    binary_list=list(arr)
    for item in binary_list:
        if item == 0:
            out.append('bad')
        else:
            out.append('good')
    return out
    
    

In [34]:
# removes pattern in the input text
def remove_pattern(input_txt, pattern):
    patterns = re.findall(pattern, input_txt)
    for word in patterns:
        input_txt = re.sub(word, "", input_txt)
    return input_txt

In [35]:
# remove twitter handles (@user)
processed_data['clean_tweet'] = np.vectorize(remove_pattern)(processed_data['tweet'], "@[\w]*")

In [36]:
processed_data.head()

,id,label,tweet,clean_tweet
0,1,1,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so sel...
1,2,1,@user @user thanks for #lyft credit i can't us...,thanks for #lyft credit i can't use cause th...
2,3,1,bihday your majesty,bihday your majesty
3,4,1,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...
4,5,1,factsguide: society now #motivation,factsguide: society now #motivation


In [37]:
# remove special characters, numbers and punctuations
processed_data['clean_tweet'] = processed_data['clean_tweet'].str.replace("[^a-zA-Z#]", " ")
processed_data.head()

<ipython-input-37-3d1c8752cad1>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  processed_data['clean_tweet'] = processed_data['clean_tweet'].str.replace("[^a-zA-Z#]", " ")


,id,label,tweet,clean_tweet
0,1,1,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so sel...
1,2,1,@user @user thanks for #lyft credit i can't us...,thanks for #lyft credit i can t use cause th...
2,3,1,bihday your majesty,bihday your majesty
3,4,1,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...
4,5,1,factsguide: society now #motivation,factsguide society now #motivation


In [38]:
# remove short words
processed_data['clean_tweet'] = processed_data['clean_tweet'].apply(lambda x: " ".join([w for w in x.split() if len(w)>3]))
processed_data.head()


,id,label,tweet,clean_tweet
0,1,1,@user when a father is dysfunctional and is s...,when father dysfunctional selfish drags kids i...
1,2,1,@user @user thanks for #lyft credit i can't us...,thanks #lyft credit cause they offer wheelchai...
2,3,1,bihday your majesty,bihday your majesty
3,4,1,#model i love u take with u all the time in ...,#model love take with time
4,5,1,factsguide: society now #motivation,factsguide society #motivation


In [39]:
# individual words considered as tokens
start_time = time.time()
tokenized_tweet = processed_data['clean_tweet'].apply(lambda x: x.split())
print(tokenized_tweet.head(5))
print('****************************')
print("-------- ",(time.time() - start_time),' Secounds --------')

0    [when, father, dysfunctional, selfish, drags, ...
1    [thanks, #lyft, credit, cause, they, offer, wh...
2                              [bihday, your, majesty]
3                     [#model, love, take, with, time]
4                   [factsguide, society, #motivation]
Name: clean_tweet, dtype: object
****************************
--------  0.05685138702392578  Secounds --------


In [40]:
# stem the words
start_time = time.time()
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

tokenized_tweet = tokenized_tweet.apply(lambda sentence: [stemmer.stem(word) for word in sentence])
tokenized_tweet.head()
print("-------- ",(time.time() - start_time),' Secounds --------')

--------  14.488245487213135  Secounds --------


In [41]:
# combine words into single sentence
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = " ".join(tokenized_tweet[i])
    
processed_data['clean_tweet'] = tokenized_tweet
processed_data.head()

,id,label,tweet,clean_tweet
0,1,1,@user when a father is dysfunctional and is s...,when father dysfunct selfish drag kid into dys...
1,2,1,@user @user thanks for #lyft credit i can't us...,thank #lyft credit caus they offer wheelchair ...
2,3,1,bihday your majesty,bihday your majesti
3,4,1,#model i love u take with u all the time in ...,#model love take with time
4,5,1,factsguide: society now #motivation,factsguid societi #motiv


#  N-Gram

In [42]:
ngram_list=list(processed_data['clean_tweet'])
ngram_list[:10]

['when father dysfunct selfish drag kid into dysfunct #run',
 'thank #lyft credit caus they offer wheelchair van #disapoint #getthank',
 'bihday your majesti',
 '#model love take with time',
 'factsguid societi #motiv',
 'huge fare talk befor they leav chao disput when they there #allshowandnogo',
 'camp tomorrow danni',
 'next school year year exam think about that #school #exam #hate #imagin #actorslif #revolutionschool #girl',
 'love land #allin #cav #champion #cleveland #clevelandcavali',
 'welcom here']

In [43]:
def bi_gram(txt):
    text=' '.join(txt)
    words=nltk.word_tokenize(text)
    _bigram=nltk.bigrams(words)
    print([' '.join(gram) for gram in _bigram])
    

In [44]:
bi_gram(ngram_list[:3])

['when father', 'father dysfunct', 'dysfunct selfish', 'selfish drag', 'drag kid', 'kid into', 'into dysfunct', 'dysfunct #', '# run', 'run thank', 'thank #', '# lyft', 'lyft credit', 'credit caus', 'caus they', 'they offer', 'offer wheelchair', 'wheelchair van', 'van #', '# disapoint', 'disapoint #', '# getthank', 'getthank bihday', 'bihday your', 'your majesti']


##  Features Extraction and Input Split

max_dffloat in range [0.0, 1.0] or int, default=1.0
When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

min_dffloat in range [0.0, 1.0] or int, default=1
When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

max_featuresint, default=None
If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.

This parameter is ignored if vocabulary is not None.

In [53]:
# feature extraction
from sklearn.feature_extraction.text import CountVectorizer

bag_of_words_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english') 
#bag_of_words_vectorizer=CountVectorizer(stop_words='english') # train all of features (take more time )
bag_of_words = bag_of_words_vectorizer.fit_transform(processed_data['clean_tweet'])

In [46]:
#print(bow[:])
#bow[0].toarray()

In [47]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(bag_of_words, processed_data['label'], random_state=42, test_size=0.25)

## Model Training

In [48]:
start_time = time.time()

from nltk.classify.scikitlearn import SklearnClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix,f1_score

# Define models to train
names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(max_iter = 100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = zip(names, classifiers)
scored_models=dict()

for name, model in models:
    nltk_model = model
    nltk_model.fit(x_train,y_train)
    pred = model.predict(x_test)
    scored_models[name]=[model,pred]
    score=f1_score(y_test, pred)
    accuracy = accuracy_score(y_test,pred) 
    print(name," Accuracy: ", accuracy," Score: ",score )
    
print("-------- ",(time.time() - start_time),' Secounds --------')

K Nearest Neighbors  Accuracy:  0.9405581278938806  Score:  0.9686943913530615
Decision Tree  Accuracy:  0.9112751845826554  Score:  0.9516535969996591
Random Forest  Accuracy:  0.9441872106119384  Score:  0.970111245141402
Logistic Regression  Accuracy:  0.9469403078463271  Score:  0.9719910159862597
SGD Classifier  Accuracy:  0.947691152546615  Score:  0.9724347137958322
Naive Bayes  Accuracy:  0.9356776373420098  Score:  0.9654523457453958
SVM Linear  Accuracy:  0.9473157301964711  Score:  0.972194703123968
--------  55.955169677734375  Secounds --------


In [49]:
scored_models

{'K Nearest Neighbors': [KNeighborsClassifier(),
  array([1, 1, 1, ..., 1, 1, 1])],
 'Decision Tree': [DecisionTreeClassifier(), array([1, 1, 1, ..., 1, 1, 1])],
 'Random Forest': [RandomForestClassifier(), array([1, 1, 1, ..., 1, 1, 1])],
 'Logistic Regression': [LogisticRegression(), array([1, 1, 1, ..., 1, 1, 1])],
 'SGD Classifier': [SGDClassifier(max_iter=100),
  array([1, 1, 1, ..., 1, 1, 1])],
 'Naive Bayes': [MultinomialNB(), array([1, 1, 0, ..., 1, 1, 1])],
 'SVM Linear': [SVC(kernel='linear'), array([1, 1, 1, ..., 1, 1, 1])]}

In [50]:
#test a specific model
test=scored_models['Random Forest'][0].predict(x_test[1])
print("binary values :",test[:10])
print('''
-------------------------------------
''')
result=decoder(test)
print('type of test :',type(x_test))
print('''
-------------------------------------
''')
print('matrix : ',x_test[1])
print('''
-------------------------------------
''')
print('test matrix shape :',x_test.shape)
print('''
-------------------------------------
''')
print(" actual labels :",result[:10])
print('''
-------------------------------------
''')
print('bad Tweets = ',result.count('bad'),'good tweets = ', result.count('good'))
print('''
-------------------------------------
''')

binary values : [1]

-------------------------------------

type of test : <class 'scipy.sparse.csr.csr_matrix'>

-------------------------------------

matrix :    (0, 205)	1
  (0, 490)	1
  (0, 541)	1
  (0, 930)	1

-------------------------------------

test matrix shape : (7991, 1000)

-------------------------------------

 actual labels : ['good']

-------------------------------------

bad Tweets =  0 good tweets =  1

-------------------------------------



In [51]:
scored_models

{'K Nearest Neighbors': [KNeighborsClassifier(),
  array([1, 1, 1, ..., 1, 1, 1])],
 'Decision Tree': [DecisionTreeClassifier(), array([1, 1, 1, ..., 1, 1, 1])],
 'Random Forest': [RandomForestClassifier(), array([1, 1, 1, ..., 1, 1, 1])],
 'Logistic Regression': [LogisticRegression(), array([1, 1, 1, ..., 1, 1, 1])],
 'SGD Classifier': [SGDClassifier(max_iter=100),
  array([1, 1, 1, ..., 1, 1, 1])],
 'Naive Bayes': [MultinomialNB(), array([1, 1, 0, ..., 1, 1, 1])],
 'SVM Linear': [SVC(kernel='linear'), array([1, 1, 1, ..., 1, 1, 1])]}

In [52]:
# print a confusion matrix and a classification report
print(classification_report(y_test, scored_models['Naive Bayes'][1]))

pd.DataFrame(
    confusion_matrix(y_test, scored_models['Naive Bayes'][1]),
    index = [['actual', 'actual'], ['bad', 'good']],
    columns = [['predicted', 'predicted'], ['bad', 'good']])


              precision    recall  f1-score   support

           0       0.54      0.53      0.53       559
           1       0.96      0.97      0.97      7432

    accuracy                           0.94      7991
   macro avg       0.75      0.75      0.75      7991
weighted avg       0.93      0.94      0.94      7991



predicted      
                  bad  good
actual bad        295   264
       good       250  7182